In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
0,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,Female,50,Bangalore,,Having a loving and caring support system . A ...,Grew up in a sports family. Mom is from Punjab...,"Was married earlier, legally divorced",Yes,Over 90 Lakhs,Currently on a career break but associated wit...,...,I am on the fence about wanting a partner 😃 I ...,I give consent,Hindu,Yes,Yes I am,Yes,Friend recommended,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
413,Male,36,Bengaluru,,Having a supportive family and dependable frie...,Growing up in a typical Indian middle-class fa...,Never married,No,Between 12-15 Lakhs,I am a visual communicator who loves working w...,...,\nI love meeting people who share similar life...,I give consent,Hindu,Yes,It will not work for me,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
414,Female,39,Mumbai,,for me privilege is to be surrounded with peo...,"It was challenging, but having my family aroun...",Never married,No,Between 12-15 Lakhs,Social Media manager,...,I am nonjudgmental to an extent and have unrea...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,https://docs.g

In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
2,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,Female,50,Bangalore,,Having a loving and caring support system . A ...,Grew up in a sports family. Mom is from Punjab...,"Was married earlier, legally divorced",Yes,Over 90 Lakhs,Currently on a career break but associated wit...,...,I am on the fence about wanting a partner 😃 I ...,I give consent,Hindu,Yes,Yes I am,Yes,Friend recommended,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
415,Male,36,Bengaluru,,Having a supportive family and dependable frie...,Growing up in a typical Indian middle-class fa...,Never married,No,Between 12-15 Lakhs,I am a visual communicator who loves working w...,...,\nI love meeting people who share similar life...,I give consent,Hindu,Yes,It will not work for me,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
416,Female,39,Mumbai,,for me privilege is to be surrounded with peo...,"It was challenging, but having my family aroun...",Never married,No,Between 12-15 Lakhs,Social Media manager,...,I am nonjudgmental to an extent and have unrea...,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,https://docs.g

In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
414,Female,50,Bangalore,"Was married earlier, legally divorced"
415,Male,36,Bengaluru,Never married
416,Female,39,Mumbai,Never married
417,Female,38,Mumbai,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Legally Separated,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
414,0,0,0,0,0,1,0
415,0,1,0,0,0,0,0
416,0,1,0,0,0,0,0
417,0,1,0,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
414,Female,50,Bangalore,"Was married earlier, legally divorced"
415,Male,36,Bengaluru,Never married
416,Female,39,Mumbai,Never married
417,Female,38,Mumbai,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
414,Female,50,Bangalore,"Was married earlier, legally divorced"
415,Male,36,Bengaluru,Never married
416,Female,39,Mumbai,Never married
417,Female,38,Mumbai,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(417, 147)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 401 109 252 409 396  11 316 108 372 272 165 342  45 266 331
  313  25 320  41 330 278 343 385 332 326 383 245 247  48 290 351 261 156
   24 311  81 229 386  26 309 358 139 181 256 118  97 363 123 354 333 288
   51 122  30 295 128 390  54 110 376 262  86 136 412  84 349 414 260 381
   19 231  75 259  55 325 274 307 114 310 336 365 393  52 200 415 308 183
   42 287 265 355 375 174  27   6   1 353  50 404  18  73 105  32 100 378
  228 327 185  29 258 251 344  12 111  65 291 221 373 282 246 178 301  66
  345 121 296 406 166  96  77 201  83 388 106 244 348  56 411 135 196 169
  237 380  36  91 194 241 143  89 379 107 209 340 328 160 357 208 163  71
  171 255 219 335 321 352 230 314 184  76 413  53 403 142 147  35 408 205
  214 263 382 173 119 211 249 238 112  21 271 104 264 242 141 137  79  70
  191  94 391  62 180 315  59  37 257   9 346 188 152 405 297 101 179 324
   23  61  60 368 117 276  57   8 361 131 293 212 319 133  46 217  10 170
  195 199 146 189  22 223  14 132 130 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 50 'Delhi ' 'Separated, not legally divorced']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Bangalore, Karnataka, India' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Widow/ Widower']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 55 'Gurgaon' 'Separated, not legally divorced']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 41 'Bengaluru' 'Never married'

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.73683683  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.23803657  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.90310359  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.26076368  0.         ...  0.          0.
   0.        ]
 [ 0.          0.09449693  0.         ...  0.          0.
   0.        ]
 [ 0.         -0.90310359  0.         ...  0.          0.
   0.        ]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']